# Math Question Answer Verification Competition

**Goal**: Fine-tune Llama-3-8B model to predict if a given solution to a math problem is correct or not.

**Try 2 Configuration (90K Samples)**:
- **81,000 training samples** + **9,000 validation samples**
- **LoRA rank 20** for good capacity
- **LoRA alpha 40** (2×rank for optimal scaling)
- **LoRA dropout 0.1** for regularization
- **Max sequence length 2048** to prevent truncation
- **1 epoch** training (fast completion)
- **Learning rate 1e-4** for stable convergence
- **Warmup 100 steps** for quick start
- **Batch size 8, Gradient accumulation 8** (effective batch size 64)
- **Improved prompt template** emphasizing solution reasoning
- **Constrained decoding** for reliable output parsing

**Note**: This configuration was tested but **not submitted** to Kaggle.

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [33]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9.]{3,}", torch.__version__).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

## **Step 2: Hugging Face Login**

For accessing gated models like Llama-3-8B, we need to authenticate with Hugging Face using your token.



In [34]:
from huggingface_hub import login

# Login to Hugging Face using token from Colab Secrets
# Set HF_TOKEN in Colab Secrets (Secrets → Add Secret)
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token, add_to_git_credential=False)
    print("✅ Successfully logged in to Hugging Face")
except:
    print("⚠️ Warning: HF_TOKEN not found. Make sure to set it in Colab Secrets if the model is gated.")

⚠️ Warning: HF_TOKEN not found. Make sure to set it in Colab Secrets if the model is gated.


## **Step 3: Load the Model and Tokenizer**

Load Llama-3-8B using Unsloth's FastLanguageModel with optimized settings:
- **4-bit quantization**: Reduces GPU memory usage significantly
- **Max sequence length 2048**: Prevents truncation of long problems
- **Auto dtype detection**: Optimizes for your GPU



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Maximum sequence length to prevent truncation
dtype = None  # Auto-detect best data type for GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Using Meta-Llama-3.1-8B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"✅ Model loaded with max_seq_length={max_seq_length}")

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model loaded with max_seq_length=2048


## **Step 4: Prepare the Dataset**

This step prepares the training data for fine-tuning. It consists of three parts:

1. **Load Dataset**: Load the full 1M training samples from Hugging Face
2. **Split Dataset**: Create train/validation split using stratified sampling (maintains class balance)
3. **Format Prompts**: Convert data into instructional prompts with improved structure

---

### 4.1 Load and Split Dataset

**Why Stratified Split?**
- Ensures training and validation sets have the same True/False ratio
- Prevents bias in validation metrics
- Better representation of the overall dataset distribution

---

### 4.2 Format Training Prompts

**Key Improvements Over Baseline:**

1. **Includes Student Answer**: The baseline prompt only had Question and Solution, but the task requires comparing the student's answer with the correct solution. This is critical!
2. **Emphasizes Reasoning**: The prompt explicitly asks the model to analyze step-by-step reasoning from the solution
3. **Clear Task Definition**: Better structure helps the model understand what it needs to do

In [ ]:
from datasets import load_dataset

# Load the full training dataset (1M samples)
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Encode is_correct as ClassLabel for stratified split
# This ensures the train/val split maintains the same True/False ratio
full_dataset = full_dataset.class_encode_column("is_correct")

# ⚡ Try 2: Limit to 90,000 samples for faster training (9% of full dataset)
full_dataset = full_dataset.shuffle(seed=42).select(range(90000))

# Stratified split: 90% training, 10% validation (maintains class balance)
# Using reasonable validation set for model evaluation
split_dataset = full_dataset.train_test_split(
    test_size=0.1,  # 10% for validation (9,000 samples)
    seed=42,
    stratify_by_column="is_correct"
)

train_dataset = split_dataset["train"]  # 81,000 samples
validation_dataset = split_dataset["test"]  # 9,000 samples

print(f"✅ Training samples: {len(train_dataset):,}")
print(f"✅ Validation samples: {len(validation_dataset):,}")
print(f"⚡ Try 2: Using 90,000 samples (9% of full dataset) for fast training")

✅ Training samples: 81,000
✅ Validation samples: 9,000
⚡ Try 1.7: Using 90,000 samples (9% of full dataset) for fast training


In [63]:
# Improved prompt template emphasizing solution reasoning
training_prompt = """You are an expert mathematician verifying student answers.

Your task: Determine if the student's answer is correct by analyzing the problem, the provided solution's reasoning steps, and the student's answer.

Question:
{}

Correct Solution (with step-by-step reasoning):
{}

Student Answer:
{}

Based on the solution's reasoning, is the student's answer correct?
Respond with:
- 'True' if the answer is correct
- 'False' if the answer is incorrect

Output:
{}"""

# EOS token to mark completion
EOS_TOKEN = tokenizer.eos_token

# Format data samples into the prompt template
def formatting_prompts_func(examples):
    """Format dataset examples into training prompts.

    Args:
        examples: Batch of dataset examples containing question, solution, answer, is_correct

    Returns:
        Dictionary with formatted text prompts
    """
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]  # Student answers
    outputs = examples["is_correct"]
    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        # Format the prompt with all components and add EOS token
        text = training_prompt.format(
            question,
            str(solution),
            str(answer),
            str(output)
        ) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply formatting to both training and validation datasets
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

print(f"✅ Datasets formatted: {len(formatted_train_dataset):,} training, {len(formatted_validation_dataset):,} validation")

Map:   0%|          | 0/81000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

✅ Datasets formatted: 81,000 training, 9,000 validation



## **Step 5: Configure LoRA**

**LoRA (Low-Rank Adaptation)** allows us to efficiently fine-tune the model by training only a small number of adapter parameters instead of the full model.

**Try 2 Settings**:
- **Rank 20**: Good capacity for 90K dataset
- **Alpha 40**: Typically set to 2×rank for optimal scaling
- **Dropout 0.1**: Regularization to prevent overfitting

In [ ]:
# Configure LoRA with Try 2 parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 20,  # LoRA rank for good capacity
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 40,  # Typically 2×rank
    lora_dropout = 0.1,  # Regularization
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

model.print_trainable_parameters()

trainable params: 52,428,800 || all params: 8,082,690,048 || trainable%: 0.6487


## **Step 6: Set Up SFTTrainer**

Configure the training process with **Try 2 Training Settings**:
- **1 epoch**: Single pass through 81K training dataset
- **Learning rate 1e-4**: Stable learning rate for good convergence
- **Warmup 100 steps**: Shorter warmup to start learning faster
- **Batch size 8**: Per-device batch size
- **Gradient accumulation 8**: Effective batch size of 64 (8 × 8)
- **No validation during training**: For maximum speed (validation checked separately)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 8,  # Try 2: Batch size 8
        gradient_accumulation_steps = 8,  # Effective batch size: 64
        warmup_steps = 100,  # Try 2: Quick warmup
        num_train_epochs = 1,  # Single epoch
        learning_rate = 1e-4,  # Try 2: Stable learning rate
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        eval_strategy = "no",  # Disable validation during training
        save_strategy = "epoch",  # Save only at end of each epoch
        save_total_limit = 2,  # Keep only 2 checkpoints
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("✅ Try 2 Trainer configured: 81K training samples, 1 epoch, LR 1e-4, Batch 8×8=64, Warmup 100")

✅ Try 1.7 Trainer configured: 9K samples, LR 1e-4, Warmup 100



## **Step 7: Start Training**

Train the model for **1 epoch** over **81,000 training samples**

In [68]:
# Start training
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 81,000 | Num Epochs = 1 | Total steps = 1,266
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 52,428,800 of 8,082,690,048 (0.65% trained)


Step,Training Loss
100,0.865600
200,0.608600
300,0.583500
400,0.565400
500,0.552400
600,0.537500
700,0.524200
800,0.508200
900,0.492200
1000,0.483900


TrainOutput(global_step=1266, training_loss=0.5521391616990028, metrics={'train_runtime': 14803.2622, 'train_samples_per_second': 5.472, 'train_steps_per_second': 0.086, 'total_flos': 2.0443349719221535e+18, 'train_loss': 0.5521391616990028, 'epoch': 1.0})

## **Step 8: Prepare for Inference**

Prepare the trained model for faster inference and test on a validation example to verify it's working correctly.


In [69]:
# Prepare model for inference
FastLanguageModel.for_inference(model)

# Improved inference prompt (matching training prompt structure)
inference_prompt = """You are an expert mathematician verifying student answers.

Your task: Determine if the student's answer is correct by analyzing the problem, the provided solution's reasoning steps, and the student's answer.

Question:
{}

Correct Solution (with step-by-step reasoning):
{}

Student Answer:
{}

Based on the solution's reasoning, is the student's answer correct?
Respond with:
- 'True' if the answer is correct
- 'False' if the answer is incorrect

Output:
"""

# Test on a validation example
example = validation_dataset[1]
question = example["question"]
solution = example["solution"]
answer = example["answer"]

# Format the prompt
inputs = tokenizer(
    [inference_prompt.format(question, str(solution), str(answer))],
    return_tensors="pt",
    truncation=True,
    max_length=max_seq_length
).to("cuda")

# Generate prediction
outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
response = tokenizer.batch_decode(outputs)[0]

# Display results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution[:200] + "..." if len(str(solution)) > 200 else solution)
print("\n#### STUDENT ANSWER ####")
print(answer)
print("\n#### MODEL'S PREDICTION ####")
output_part = response.split("Output:\n")[-1]
print(output_part[:50])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
Find the product of $218_9 \cdot 5_9$.  Express your answer in base 9.

#### SOLUTION ####
We will first convert the numbers to their decimal equivalents, then find the product using sympy, and finally convert the result back to base $9$.
<llm-code>
import sympy as sp

# convert the numbers...

#### STUDENT ANSWER ####
12.7448559670782

#### MODEL'S PREDICTION ####
0<|end_of_text|>

#### CORRECT ANSWER ####
0


In [74]:
# Prepare model for inference
FastLanguageModel.for_inference(model)

# Improved inference prompt (matching training prompt structure)
inference_prompt = """You are an expert mathematician verifying student answers.

Your task: Determine if the student's answer is correct by analyzing the problem, the provided solution's reasoning steps, and the student's answer.

Question:
{}

Correct Solution (with step-by-step reasoning):
{}

Student Answer:
{}

Based on the solution's reasoning, is the student's answer correct?
Respond with:
- 'True' if the answer is correct
- 'False' if the answer is incorrect

Output:
"""
count = 0

# Test on a all validation examples
for i in range(len(validation_dataset)):
  example = validation_dataset[i]
  question = example["question"]
  solution = example["solution"]
  answer = example["answer"]

  # Format the prompt
  inputs = tokenizer(
      [inference_prompt.format(question, str(solution), str(answer))],
      return_tensors="pt",
      truncation=True,
      max_length=max_seq_length
  ).to("cuda")

  # Generate prediction
  outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
  response = tokenizer.batch_decode(outputs)[0]

  if response.split("Output:\n")[-1].split("<")[0] == '':
    continue

  if example["is_correct"] == int(response.split("Output:\n")[-1].split("<")[0]):
    count += 1

# Compute the prediction accuracy on the validation examples
print(f"Accuracy: {count/len(validation_dataset)}")

Accuracy: 0.8292222222222222


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import LogitsProcessor, GenerationConfig
import torch

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

questions = []
solutions = []
answers = []

for i, example in enumerate(tqdm(test_dataset)):
    question = example["question"]
    solution = example["solution"]
    answer = example["answer"]

    questions.append(question)
    solutions.append(solution)
    answers.append(answer)

# Create a DataFrame
df = pd.DataFrame({
    'Question': questions,
    'Solution': solutions,
    'Answer': answers
})

df.to_csv('test_data.csv', index=False)

print(f"Test data saved to 'test_data.csv'")

100%|██████████| 10000/10000 [00:00<00:00, 23820.86it/s]


Test data saved to 'test_data.csv'


## **Step 9: Generate Submission File**

Generate predictions for all test samples and create the submission CSV file.

In [71]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import LogitsProcessor, GenerationConfig
import torch

# Constrained Decoding: Force model to generate only "True"/"False" or "1"/"0" tokens
class AllowedTokensLogitsProcessor(LogitsProcessor):
    """Logits Processor that forces model to generate only allowed tokens"""
    def __init__(self, allowed_token_ids):
        self.allowed = set(allowed_token_ids)

    def __call__(self, input_ids, scores):
        # Set probability of disallowed tokens to -inf
        mask = torch.full_like(scores, float("-inf"))
        for tid in self.allowed:
            if tid < scores.shape[-1]:
                mask[..., tid] = 0.0
        return scores + mask

def get_allowed_token_ids(tokenizer):
    """Return token IDs for allowed tokens (True/False or 1/0)"""
    tokens = ["True", "False", "1", "0"]
    ids = []
    for token in tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id != tokenizer.unk_token_id:
            ids.append(token_id)
    return ids

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Setup constrained decoding
allowed_ids = get_allowed_token_ids(tokenizer)
logits_proc = AllowedTokensLogitsProcessor(allowed_ids)

# Optimized Generation configuration
gen_config = GenerationConfig(
    max_new_tokens=1,        # Reduced from 8 to 1 (faster and more accurate)
    do_sample=False,          # Deterministic generation
    temperature=0.0,          # Use probability distribution as-is
    top_p=1.0,
    eos_token_id=[tokenizer.eos_token_id],
    pad_token_id=tokenizer.eos_token_id,
)

# Generate predictions for all test samples
print(f"Generating predictions for {len(test_dataset):,} test samples...")
allowed_tokens = [tokenizer.convert_ids_to_tokens(tid) for tid in allowed_ids if tid != tokenizer.unk_token_id]
print(f"Using constrained decoding with allowed tokens: {allowed_tokens}")

for i, example in enumerate(tqdm(test_dataset)):
    question = example["question"]
    solution = example["solution"]
    answer = example["answer"]  # Student answer

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution), str(answer))
    inputs = tokenizer(
        [prompt],
        return_tensors="pt",
        truncation=True,
        max_length=max_seq_length
    ).to("cuda")

    # Generate prediction with constrained decoding
    outputs = model.generate(
        **inputs,
        generation_config=gen_config,
        logits_processor=[logits_proc],  # Apply constrained decoding
        use_cache=True
    )

    # Decode only newly generated tokens (more accurate parsing)
    new_tokens = outputs[0, inputs["input_ids"].shape[1]:]
    response_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Check first character only (0 or 1, True/False)
    ch = response_text.strip()[:1] if response_text.strip() else "0"

    # Parse: "1" or first letter "T" of "True" means True
    if ch.lower() in ["1", "t"]:
        prediction = True
    else:
        prediction = False

    predictions.append(prediction)

# Create submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print(f"\n✅ Submission file created successfully!")
print(f"   Total predictions: {len(predictions):,}")
print(f"   True predictions: {sum(predictions):,}")
print(f"   False predictions: {len(predictions) - sum(predictions):,}")
print("\n📁 File saved as 'submission.csv'")
print("   You can now download this file and submit it to the Kaggle competition.")

Generating predictions for 10,000 test samples...
Using constrained decoding with allowed tokens: ['True', 'False', '1', '0']


100%|██████████| 10000/10000 [32:40<00:00,  5.10it/s]



✅ Submission file created successfully!
   Total predictions: 10,000
   True predictions: 3,570
   False predictions: 6,430

📁 File saved as 'submission.csv'
   You can now download this file and submit it to the Kaggle competition.
